### Load Packages

In [35]:
import numpy as np
import pandas as pd
import os
from datetime import datetime, timedelta

### Remove unwanted characters from Columns

In [36]:
# Remove unwanted characters from Columns
def renamer(df):
    drive = df.copy()
    drive.columns = drive.columns.str.replace("[", "_")
    drive.columns = drive.columns.str.replace("]","")
    drive.columns = drive.columns.str.replace("(", "_")
    drive.columns = drive.columns.str.replace(")","")
    drive.columns = drive.columns.str.replace("/","")
    drive.columns = drive.columns.str.replace("%","Pctg")
    
    return drive

### Calculations

In [37]:
# Add column with speed as m_s
# drive['Vehicle Speed_m_s'] = drive['Vehicle Speed_kmh']/3.6
# Add column with Acceleration
# drive['Vehicle Acceleration'] = drive['Vehicle Speed_m_s'] - drive['Vehicle Speed_m_s'].shift()
# Add Column with Jerk
# drive['Vehicle Jerk'] = drive['Vehicle Acceleration'] - drive['Vehicle Acceleration'].shift()
# Add column with distance travelled in that second
# drive['Dist_in_s'] = drive['Vehicle Speed_m_s']/1609.344
# Add column with cumulative distance
# drive['Cumsum_Dist'] = drive['Dist_in_s'].cumsum()
# Consumption for every second
# drive['kwh'] = ((drive['HV Battery Current_A'] - drive['HV Battery Voltage_V']/1000)/3600))

### Create list with all raw data input files

In [38]:
for filename in os.listdir('/Users/timjonas/ElectricVehicle/RawDataComplete'):
    if filename.endswith(".csv"): 
        print(filename)
        # drive = pd.read_csv(filename, index_col=0)
        # Calculations
        # calculations.to_csv('output_dir/'+fn)
        
def getFiles():
    fileList = []
    for filename in os.listdir('/Users/timjonas/ElectricVehicle/RawDataComplete'):
        if filename.endswith(".csv"):
            fileList.append(filename)
    
    return fileList


2018-11-29T20-34-18.csv
2018-11-30T15-03-05.csv
2018-11-14T12-52-06.csv
2018-11-27T15-53-11.csv
2018-11-05T21-05-26.csv
2018-11-19T13-01-20_run2.csv
2018-11-27T16-49-43_run2.csv
2018-11-14T19-54-58.csv
2018-11-19T13-01-20_run1.csv
2018-11-27T16-49-43_run1.csv
2018-11-08T15-16-59.csv
2018-11-15T16-47-06.csv
2018-12-04T12-46-28_run1.csv
2018-12-04T12-46-28_run2.csv
2018-11-15T12-47-44.csv
2018-11-08T13-19-45.csv
2018-11-15T13-37-51.csv
2018-11-28T12-26-14.csv
2018-11-14T20-49-28.csv
2018-11-07T19-48-33.csv
2018-11-28T13-17-30.csv
2018-11-07T13-22-03.csv
2018-11-21T15-57-44.csv
2018-11-26T12-34-09.csv
2018-11-29T12-35-21.csv
2018-11-30T15-54-22.csv
2018-11-14T13-37-37.csv
2018-11-21T13-08-24.csv
2018-11-26T19-23-11.csv
2018-11-29T19-33-55.csv


### Loop through all files in the input folder and do all calculations

### Create file with summary data

In [39]:
columns = ['Workbookname','Date','Time','Type','Total Consumption (kWh)','Distance','Mean Consumption','Total Time (ms)','Average Speed','StdDev Speed','Average Acceleration','StdDev Acceleration','Average Jerk','StdDev Jerk','Initial SOC','Temperature']
TotalDrive = pd.DataFrame(columns = columns)

In [40]:
fileList = getFiles()


for i in range(len(fileList)):
    drive = pd.read_csv('../ElectricVehicle/RawDataComplete'+'/'+fileList[i], skiprows= 5, header=0)
    drive = renamer(drive)
    drive = drive[drive.Longitude_deg != 0]
    print(i)
    # Calculations
    # Add column with speed as m_s
    drive['Vehicle Speed_m_s'] = drive['Vehicle Speed_kmh']/3.6
    # Add column with Acceleration
    drive['Vehicle Acceleration'] = drive['Vehicle Speed_m_s'] - drive['Vehicle Speed_m_s'].shift()
    # Add Column with Jerk
    drive['Vehicle Jerk'] = drive['Vehicle Acceleration'] - drive['Vehicle Acceleration'].shift()
    # Add column with distance travelled in that second
    drive['Dist_in_s'] = drive['Vehicle Speed_m_s']/1609.344
    # Add column with cumulative distance
    drive['Cumsum_Dist'] = drive['Dist_in_s'].cumsum()
    # Consumption for every second
    drive['kwh'] = ((drive['HV Battery Current_A'] * drive['HV Battery Voltage_V']*-1)/1000/3600)
    # Add column with distance to start point
    drive['Start_Cut'] = ((drive['Latitude_deg']-41.489207)**2)+((drive['Longitude_deg']+71.521673)**2)
    drive['Start_Cut']=np.sqrt((drive['Start_Cut']))
    # Add column with distance to end point
    drive['End_Cut'] = ((drive['Latitude_deg']-41.480337)**2)+((drive['Longitude_deg']+71.525495)**2)
    drive['End_Cut']=np.sqrt((drive['End_Cut']))
    drive.truncate(drive['Start_Cut'].idxmin(),drive['End_Cut'].idxmin())
    
    # Correcting and adding Date/Hour
    # Correct format for Date and Hour
    A = fileList[i] # At first is a str
    A = A.replace('T',' ')
    B = A[0:11]
    C = A[11:19]
    C = C.replace('-',':')
    D = B+C

    DateHour = datetime.strptime(D, "%Y-%m-%d %H:%M:%S") # Convert to datetime
    Date = DateHour.strftime('%Y-%m-%d') # Split only the Date and convert to str again
    CorrectHour = DateHour - timedelta(hours=5) # Minus 4 hours
    Hour = CorrectHour.strftime('%H:%M:%S') # Split only the Date and convert to str again
    
    hour = datetime.strptime(Hour,'%H:%M:%S')
    InicioTraffic = datetime.strptime("07:00:00", "%H:%M:%S")
    FinTraffic = datetime.strptime("09:00:00", "%H:%M:%S")
 
    if hour >= InicioTraffic and hour <= FinTraffic:
        Type = "Traffic"
    else:
        Type = "Non Traffic"

    
    TotalDrive = TotalDrive.append({'Workbookname' : fileList[i], 'Date' : Date, 'Time' : Hour, 'Type' : Type, 'Total Consumption (kWh)' : drive['kwh'].sum(), 'Distance' : drive.iloc[-1]['Cumsum_Dist'], 'Mean Consumption' : drive['kwh'].mean(), 'Average Speed' : drive['Vehicle Speed_kmh'].mean(), 'StdDev Speed' : drive['Vehicle Speed_kmh'].std(), 'Average Acceleration' : drive['Vehicle Acceleration'].mean(), 'StdDev Acceleration' : drive['Vehicle Acceleration'].std(), 'Average Jerk' : drive['Vehicle Jerk'].mean(), 'StdDev Jerk' : drive['Vehicle Jerk'].std(), 'Total Time (ms)' : drive.iloc[-1]['Timestamp_ms'] - drive.iloc[0]['Timestamp_ms'], 'Initial SOC' : drive.iloc[0]['HV Battery SOC_Pctg'], 'Temperature' : drive['OAT_degC'].mean()} , ignore_index=True)
    # drive.to_csv('../ElectricVehicle/OutputFiles'+'/'+fileList[i])

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29


In [41]:
TotalDrive.head(30)

,Workbookname,Date,Time,Type,Total Consumption (kWh),Distance,Mean Consumption,Total Time (ms),Average Speed,StdDev Speed,Average Acceleration,StdDev Acceleration,Average Jerk,StdDev Jerk,Initial SOC,Temperature
0,2018-11-29T20-34-18.csv,2018-11-29,15:34:18,Non Traffic,7.074403,27.745432,0.002197,3219000.0,49.921429,31.148579,-1.725864e-04,0.822531,-0.000086,0.635635,39.0,4.407453
1,2018-11-30T15-03-05.csv,2018-11-30,10:03:05,Non Traffic,6.413572,26.856008,0.002341,2739000.0,56.786131,23.158365,-3.042473e-04,0.836961,-0.000406,0.742772,72.5,2.978102
2,2018-11-14T12-52-06.csv,2018-11-14,07:52:06,Traffic,8.178843,26.867745,0.003036,2693000.0,57.780995,26.117368,-2.062962e-04,0.985821,-0.000825,0.796727,97.5,1.449889
3,2018-11-27T15-53-11.csv,2018-11-27,10:53:11,Non Traffic,6.653620,27.075041,0.002521,2638000.0,59.440318,23.215887,-3.158959e-04,0.803989,0.000000,0.646123,93.0,4.773020
4,2018-11-05T21-05-26.csv,2018-11-05,16:05:26,Non Traffic,6.672848,26.809923,0.002237,2982000.0,52.070734,26.537751,0.000000e+00,0.739233,0.000000,0.590660,79.0,10.187730
5,2018-11-19T13-01-20_run2.csv,2018-11-19,08:01:20,Traffic,7.606625,28.584110,0.002528,3008000.0,55.036889,31.425671,-2.123966e-03,1.015736,0.000462,0.825395,49.5,6.322366
6,2018-11-27T16-49-43_run2.csv,2018-11-27,11:49:43,Non Traffic,6.936786,28.145181,0.002420,2865000.0,56.895673,25.913440,-3.878224e-03,0.878525,0.000000,0.696426,50.0,5.844731
7,2018-11-14T19-54-58.csv,2018-11-14,14:54:58,Non Traffic,8.133592,28.496601,0.002551,3188000.0,51.771402,30.066772,-1.114402e-18,0.878934,-0.000349,0.733942,100.0,0.532769
8,2018-11-19T13-01-20_run1.csv,2018-11-19,08:01:20,Traffic,7.387900,28.340222,0.002491,2965000.0,55.358395,27.051709,1.405284e-03,0.893877,-0.000375,0.727720,71.0,5.083951
9,2018-11-27T16-49-43_run1.csv,2018-11-27,11:49:43,Non Traffic,6.481990,26.849967,0.002525,2566000.0,60.599533,22.979288,4.546635e-03,0.781361,0.000108,0.673680,71.0,5.625633
